In [27]:
import tools
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
print(os.getenv("LANGSMITH_ENDPOINT"))

https://api.smith.langchain.com


In [28]:
import os

import os
from langchain.chat_models import init_chat_model



llm = init_chat_model(
    "azure_openai:gpt-4.1-mini",
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    api_version="2024-12-01-preview"
)



In [29]:
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "ocits"
#llm.invoke("Hello world")

In [30]:
import json
base_prompt = 'Antworte in JSON und folge dem Schema: { "type": "object", "properties": { "response": { "type": "boolean" }, "reasoning": { "type": "string" } }, "required": [ "response", "reasoning" ] }'
with open("datasets/dataset_24.json", "r") as file:
    dataset = json.load(file)


responses = []
for data in dataset:
    sentence = data["sentence"]
    prompt = f"{sentence}\n{base_prompt}"
    response = llm.invoke(prompt)
    cleaned_response = response.content.strip("```json").strip("```").strip()
    response_json = json.loads(cleaned_response)
    responses.append({
        "id": data["id"],
        "plant": data["plant"],
        "city": data["city"],
        "date": data["date"],
        "sentence": sentence,
        "response": response_json["response"],
        "reasoning": response_json["reasoning"],
        "tokens": {
            "completion_tokens": response.response_metadata["token_usage"]["completion_tokens"],
            "prompt_tokens": response.response_metadata["token_usage"]["prompt_tokens"],
            "total_tokens": response.response_metadata["token_usage"]["total_tokens"]
        }
    })

print(responses)

with open("results/only_llm_24.json", "w") as file:
    json.dump(responses, file, indent=4)



[{'id': 0, 'plant': 'Apfel', 'city': 'Berlin', 'date': '11.04.2021', 'sentence': 'Heute, am 11.04.2021, befinden wir uns in Berlin. Ist die Apfel gerade am Blühen?', 'response': True, 'reasoning': 'In Berlin, Apfelbäume (Malus domestica) blühen typischerweise im April. Mitte April, einschließlich dem 11.04.2021, ist die Zeit der Apfelblüte in der Regel erreicht oder gerade im Gange, da die Temperaturen und Tageslichtverhältnisse im Frühling dies fördern.', 'tokens': {'completion_tokens': 87, 'prompt_tokens': 93, 'total_tokens': 180}}, {'id': 1, 'plant': 'Apfel', 'city': 'München', 'date': '05.04.2021', 'sentence': 'Stellen wir uns vor, es ist der 05.04.2021 und wir sind in München. Blüht die Apfel jetzt?', 'response': True, 'reasoning': 'Am 05.04.2021 in München befindet sich der Apfelbaum typischerweise in der Blütephase. In Mitteleuropa blühen Apfelbäume normalerweise von Ende März bis Mitte April, abhängig von den Witterungsbedingungen im Frühjahr. Da München ein gemäßigtes Klima ha

In [31]:
response.response_metadata["token_usage"]["completion_tokens"]
response.response_metadata["token_usage"]["prompt_tokens"]
response.response_metadata["token_usage"]["total_tokens"]
#json.loads(response.content.strip("```json").strip("```").strip())

207